In [42]:
from copy import deepcopy
import os
import numpy as np
import sncosmo

In [43]:
from LSSTmetrics import PerSNMetric
import gedankenLSST

In [44]:
from lsst.sims.photUtils import BandpassDict

In [45]:
lsst_bp = BandpassDict.loadTotalBandpassesFromFiles()

In [46]:
dailyCadence = deepcopy(gedankenLSST.LSSTReq)

In [47]:
# sncosmo Bandpasses required for fitting
throughputsdir = os.getenv('THROUGHPUTS_DIR')

from astropy.units import Unit
bandPassList = ['u', 'g', 'r', 'i', 'z', 'y']
banddir = os.path.join(os.getenv('THROUGHPUTS_DIR'), 'baseline')

for band in bandPassList:

    # setup sncosmo bandpasses
    bandfname = banddir + "/total_" + band + '.dat'


    # register the LSST bands to the SNCosmo registry
    # Not needed for LSST, but useful to compare independent codes
    # Usually the next two lines can be merged,
    # but there is an astropy bug currently which affects only OSX.
    numpyband = np.loadtxt(bandfname)
    print band
    sncosmoband = sncosmo.Bandpass(wave=numpyband[:, 0],
                                   trans=numpyband[:, 1],
                                   wave_unit=Unit('nm'),
                                   name=band)
    sncosmo.registry.register(sncosmoband, force=True)

u
g
r
i
z
y


In [7]:
#Now create a set of observations at a particular ra, dec within a a time window
observations = gedankenLSST.GSN_Obs(mjd_center=49570., 
                                    lsstrequirements=dailyCadence,
                                    ra=58., dec=-27.,
                                    timeWindow=[-30., 50.])

##  PerSNMetric: obtaining the State

In [48]:
q = PerSNMetric(summarydf=observations.summary, t0=49570,  raCol='ra', decCol='dec', lsst_bp=lsst_bp)

In [51]:
print q.SN

source:
  class      : SALT2Source
  name       : 'salt2-extended'
  version    : 1.0
  phases     : [-20, .., 50] days
  wavelengths: [300, .., 18000] Angstroms
effect (name='host' frame='rest'):
  class           : OD94Dust
  wavelength range: [909.09, 33333.3] Angstroms
effect (name='mw' frame='obs'):
  class           : OD94Dust
  wavelength range: [909.09, 33333.3] Angstroms
parameters:
  z       = 0.5
  t0      = 49570.0
  x0      = 1.0068661711630977e-05
  x1      = 0.0
  c       = 0.0
  hostebv = 0.0
  hostr_v = 3.1000000000000001
  mwebv   = 0.0
  mwr_v   = 3.1000000000000001


In [53]:
q.SN.SNstate

{'MWE(B-V)': 0.053107433021068573,
 'ModelSource': 'salt2-extended',
 '_dec': -27.0,
 '_ra': 58.0,
 'c': 0.0,
 'hostebv': 0.0,
 'hostr_v': 3.1000000000000001,
 'mwebv': 0.0,
 'mwr_v': 3.1000000000000001,
 't0': 49570.0,
 'x0': 1.0068661711630977e-05,
 'x1': 0.0,
 'z': 0.5}

In [54]:
model = sncosmo.Model(source='salt2')

In [60]:
model.set(**q.SN.sncosmoParamDict(q.SN.SNstate, model))

In [63]:
snState = deepcopy(q.SN.SNstate)

In [62]:
model.set(z=0.7)
model.set_source_peakabsmag(-19.5, 'bessellB', 'ab')

In [64]:
snState['z'] = model.get('z')

In [65]:
snState['x0'] = model.get('x0')

In [66]:
print snState

{'c': 0.0, 'x1': 0.0, 'mwebv': 0.0, 'ModelSource': 'salt2-extended', '_dec': -27.0, 't0': 49570.0, '_ra': 58.0, 'hostebv': 0.0, 'mwr_v': 3.1000000000000001, 'x0': 5.3340892341685985e-06, 'z': 0.69999999999999996, 'hostr_v': 3.1000000000000001, 'MWE(B-V)': 0.053107433021068573}


In [67]:
newQM = PerSNMetric(summarydf=observations.summary, t0=49570,  raCol='ra', decCol='dec', lsst_bp=lsst_bp, snState=snState)

In [68]:
print newQM.snState

{'c': 0.0, 'x1': 0.0, 'mwebv': 0.0, 'ModelSource': 'salt2-extended', '_dec': -27.0, 't0': 49570.0, '_ra': 58.0, 'hostebv': 0.0, 'mwr_v': 3.1000000000000001, 'x0': 5.3340892341685985e-06, 'z': 0.69999999999999996, 'hostr_v': 3.1000000000000001, 'MWE(B-V)': 0.053107433021068573}
